<a href="https://colab.research.google.com/github/chibokocl/chibokocl/blob/main/Heart_Disease_Prediction_(95_Accuracy_%26_Recall).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'heart-failure-prediction:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F1582403%2F2603715%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240521%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240521T040025Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D105583285badff6ad4993fd7387c9e535da46f8a15016dbedc4a34abed45612945a72ea78d5389b984d917df6b36e5b9da76e29d56ef4dd817421bb28906ed68b4ed5a224c607948c343d3c285b3ad0579c42e9b671218d776bbece575b8f9ec3c8b799d71e75b9465061b10694b48b2967aa7e2700524bccd0837e5c1633aa0357e5ae0e1884d4e3b0f4451b2f44c3dfb98911fc76c96e0cf8356ca89d32006ddd93b485357309c12347bb89fd0dce312bfbcd0977de06214bf83a1d85fcde795929f40e15f59b3353275bc58293be22439d66e512f99f7cdbc336c87e901cd8c2e13c950c14e706d3fc4630d89f6424249eaeb5b70ebad382e30569204cc49'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


In [ ]:
import pandas as pd
from datetime import datetime
import numpy as np
from sklearn.metrics import plot_confusion_matrix
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler,OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.ensemble import RandomForestRegressor
from numpy import loadtxt
from xgboost import XGBClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from tensorflow import keras
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.metrics import classification_report, recall_score,accuracy_score
from keras.models import Sequential
from keras import Input
from sklearn.metrics import confusion_matrix
import seaborn as sns
import tensorflow as tf
from keras.utils.vis_utils import plot_model
from keras.layers import Dense, SimpleRNN
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.layers import LSTM
from sklearn.metrics import roc_curve,auc,roc_auc_score

In [ ]:
data=pd.read_csv('../input/heart-failure-prediction/heart.csv')

In [ ]:
data.head()

## EDA or Data Preprocessing

In [ ]:
data.isnull().sum()

In [ ]:
lbl=LabelEncoder()
categorical=['Sex','ChestPainType','RestingECG','ExerciseAngina','ST_Slope']
for i in categorical:
  data[i]=lbl.fit_transform(data[i])

In [ ]:
data

In [ ]:
y=data['HeartDisease']
X=data.drop('HeartDisease',axis=1)


In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

In [ ]:
scaler = MinMaxScaler()
X_train=scaler.fit_transform(X_train)
X_test=scaler.fit_transform(X_test)


## The Gaussian Naive Bytes Classifier

In [ ]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
gnb.fit(X_train,y_train)
y_pred=gnb.predict(X_test)
gnb.score(X_test,y_test)
plot_confusion_matrix(gnb, X_test, y_test)
plt.show()

In [ ]:
print("The accuracy for Naive Bytes= "+str(accuracy_score(y_test,y_pred)))
print("Recall: "+str(recall_score(y_test, y_pred)))

## The Logistic Regression

In [ ]:
logisticRegressionmodel=LogisticRegression()
logisticRegressionmodel.fit(X_train,y_train)
y_pred=logisticRegressionmodel.predict(X_test)
plot_confusion_matrix(logisticRegressionmodel, X_test, y_test)
plt.show()

In [ ]:
print("The accuracy for Logsitic Regresion= "+str(accuracy_score(y_test,y_pred)*100))
print("Recall: "+str(recall_score(y_test, y_pred)*100))

## KNN algorithm

In [ ]:
score=[]
for i in range(1,50):
    knnalgo=KNeighborsClassifier(n_neighbors=i)
    knnalgo.fit(X_train,y_train)
    y_pred=knnalgo.predict(X_test)
    temp=mean_squared_error(y_test,y_pred)
    score.append(temp)

knnalgo=KNeighborsClassifier(n_neighbors=26)
knnalgo.fit(X_train, y_train)
y_pred = knnalgo.predict(X_test)
temp=mean_squared_error(y_test, y_pred)
plot_confusion_matrix(knnalgo, X_test, y_test)
plt.show()

In [ ]:
print("The accuracy for Naive Bytes= "+str(accuracy_score(y_test,y_pred)*100))
print("Recall: "+str(recall_score(y_test, y_pred)*100))

## RandomForestRegressor

In [ ]:
regressor = RandomForestRegressor(n_estimators = 75,max_depth=4)
regressor.fit(X_train,y_train)
y_pred=regressor.predict(X_test)
y_pred=(y_pred>0.5)

y_test=list(y_test)
y_pred=list(y_pred)


pred_train=regressor.predict(X_train)
pred_train=(pred_train>0.5)

print("The accuracy for Random Forest Regressor= "+str(accuracy_score(y_train,pred_train)*100))
print("Recall: "+str(recall_score(y_train, pred_train)*100))

print("The Test accuracy for Random Forest Regressor= "+str(accuracy_score(y_test,y_pred)*100))
print("Test Recall: "+str(recall_score(y_test, y_pred)*100))

In [ ]:
conf_train=confusion_matrix(y_train,pred_train)
print("Accuracy: "+str(accuracy_score(y_train,pred_train)*100))
print("Recall: "+str(recall_score(y_train, pred_train)*100))
ax = sns.heatmap(conf_train/np.sum(conf_train), annot=True,
            fmt='.2%', cmap='Blues')

ax.set_title('Train Data');
ax.set_xlabel('\nPredicted Values')
ax.set_ylabel('Actual Values ');

ax.xaxis.set_ticklabels(['False','True'])
ax.yaxis.set_ticklabels(['False','True'])

plt.show()

In [ ]:
conf_test=confusion_matrix(y_test,y_pred)
print("Accuracy: "+str(accuracy_score(y_test,y_pred)*100))
print("Recall: "+str(recall_score(y_test, y_pred)*100))
ax = sns.heatmap(conf_test/np.sum(conf_test), annot=True,
            fmt='.2%', cmap='Blues')

ax.set_title('Test Data\n\n');
ax.set_xlabel('\nPredicted Values')
ax.set_ylabel('Actual Values ');

## Ticket labels - List must be in alphabetical order
ax.xaxis.set_ticklabels(['False','True'])
ax.yaxis.set_ticklabels(['False','True'])

## Display the visualization of the Confusion Matrix.
plt.show()

In [ ]:
false_positive_rate1, true_positive_rate1, threshold1 = roc_curve(y_test, y_pred)
auc=roc_auc_score(y_test,y_pred)
print(auc)
plt.subplots(1, figsize=(10,10))
plt.title('ROC Curve for Heart Disease Predictor')
plt.plot(false_positive_rate1, true_positive_rate1)
plt.plot([0, 1], ls="--")
plt.plot([0, 0], [1, 0] , c=".7"), plt.plot([1, 1] , c=".7")
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()

## ANN Model

In [ ]:
annmodel = Sequential([Dense(units=4,input_shape=(11,),activation='relu'),
                    Dense(units=6,activation='relu'),
                    keras.layers.Dropout(0.3),
                    Dense(units=8,activation='relu'),
                    keras.layers.Dropout(0.3),
                    Dense(units=1,activation='sigmoid')])

In [ ]:
annmodel.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [ ]:
annmodel.fit(X_train, y_train,
          batch_size=1,
          epochs=100,
          verbose='auto')

In [ ]:
pred_train = annmodel.predict(X_train)
pred_test = annmodel.predict(X_test)
pred_train = (pred_train > 0.5)
pred_test = (pred_test > 0.5)

print("")
print('-------------------- Model Summary --------------------')
annmodel.summary()
print("")
print('---------- Evaluation on Training Data ----------')
print("Accuracy (on train data): ", accuracy_score(y_train, pred_train)*100)
print("")
print("Recall (on train data): ", recall_score(y_train, pred_train)*100)
print("")


print('---------- Evaluation on Test Data ----------')
print("Accuracy (on test data):  ", accuracy_score(y_test, pred_test)*100)
print("")
print("Recall (on test data):  ", recall_score(y_test, pred_test)*100)
print("")

In [ ]:
conf_train=confusion_matrix(y_train,pred_train)
print("Accuracy: "+str(accuracy_score(y_train,pred_train)*100))
print("Recall: "+str(recall_score(y_train, pred_train)*100))
ax = sns.heatmap(conf_train/np.sum(conf_train), annot=True,
            fmt='.2%', cmap='Blues')

ax.set_title('Train Data');
ax.set_xlabel('\nPredicted Values')
ax.set_ylabel('Actual Values ');

ax.xaxis.set_ticklabels(['False','True'])
ax.yaxis.set_ticklabels(['False','True'])

plt.show()

In [ ]:
conf_test=confusion_matrix(y_test,pred_test)
print("Accuracy: "+str(accuracy_score(y_test,pred_test)*100))
print("Recall: "+str(recall_score(y_test, pred_test)*100))
ax = sns.heatmap(conf_test/np.sum(conf_test), annot=True,
            fmt='.2%', cmap='Blues')

ax.set_title('Test Data\n\n');
ax.set_xlabel('\nPredicted Values')
ax.set_ylabel('Actual Values ');

## Ticket labels - List must be in alphabetical order
ax.xaxis.set_ticklabels(['False','True'])
ax.yaxis.set_ticklabels(['False','True'])

## Display the visualization of the Confusion Matrix.
plt.show()

In [ ]:
false_positive_rate1, true_positive_rate1, threshold1 = roc_curve(y_test, pred_test)
auc=roc_auc_score(y_test,pred_test)
print(auc)
plt.subplots(1, figsize=(10,10))
plt.title('ROC Curve for Heart Disease Predictor')
plt.plot(false_positive_rate1, true_positive_rate1)
plt.plot([0, 1], ls="--")
plt.plot([0, 0], [1, 0] , c=".7"), plt.plot([1, 1] , c=".7")
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()

## Conclusion

#### A lot of different ML/ Neural Network models have been showcased.
#### Important metrics like Recall and Accuracy are shown.

## If you liked my work, Please upvote it.